In [1]:
import numpy as np
from astropy.visualization import ZScaleInterval, PercentileInterval
from ipywidgets import interact

import bokeh
from bokeh.palettes import viridis, Category10
from bokeh.plotting import figure, output_file, show, output_notebook, save
from bokeh.models import GlyphRenderer, LinearColorMapper
from bokeh.models import LinearAxis, Range1d
from bokeh.io import push_notebook
from bokeh.layouts import gridplot, layout

#from numba import jit, njit

bokeh.__version__

'1.0.2'

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
# prepare some data
x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
y0 = [i**2 for i in x]
y1 = [10**i for i in x]
y2 = [10**(i**2) for i in x]

# output to static HTML file
#output_file("log_lines.html")

# create a new plot
p = figure(
    width=600, plot_height=400,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log", y_range=[0.001, 10**11], title="log axis example",
   x_axis_label='sections', y_axis_label='particles'
)

# add some renderers
p.line(x, x, legend="y=x")
p.circle(x, x, legend="y=x", fill_color="white", size=8)
p.line(x, y0, legend="y=x^2", line_width=3)
p.line(x, y1, legend="y=10^x", line_color="red")
p.circle(x, y1, legend="y=10^x", fill_color="red", line_color="red", size=6)
p.line(x, y2, legend="y=10^x^2", line_color="orange", line_dash="4 4")

# show the results
show(p)

In [4]:
# prepare some data
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# output to static HTML file
#output_file("linked_panning.html")

# create a new plot
s1 = figure(width=250, plot_height=250, title=None)
s1.circle(x, y0, size=10, color="navy", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(width=250, height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.triangle(x, y1, size=10, color="firebrick", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(width=250, height=250, x_range=s1.x_range, title=None)
s3.square(x, y2, size=10, color="olive", alpha=0.5)

# NEW: put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None)

# show the results
show(p)

In [5]:
import scipy.misc
img_blur = (scipy.misc.ascent()[::-1,:]/255.0)[:250, :250].copy(order='C')

In [6]:
palette = ['#%02x%02x%02x' %(i,i,i) for i in range(256)]
width, height = img_blur.shape
p_blur = figure(x_range=(0, width), y_range=(0, height))
r_blur = p_blur.image(image=[img_blur], x=[0], y=[0], dw=[width], dh=[height], palette=palette, name='blur')

In [7]:
#from numba import jit, njit

In [8]:
#@njit
def blur(outimg, img, amt):
    iw, ih = img.shape
    for i in range(amt, iw-amt):
        for j in range(amt, ih-amt):
            px = 0.
            for w in range(-amt//2, amt//2):
                for h in range(-amt//2, amt//2):
                    px += img[i+w, j+h]
            outimg[i, j]= px/(amt*amt)

In [9]:
from timeit import default_timer as timer

def update(i=0):
    level = 2*i + 1
    
    out = img_blur.copy()
    
    ts = timer()
    blur(out, img_blur, level)
    te = timer()
    print('blur takes:', te - ts)
    
    renderer = p_blur.select(dict(name="blur", type=GlyphRenderer))
    r_blur.data_source.data['image'] = [out]
    push_notebook(handle=t_blur)

In [10]:
t_blur = show(p_blur, notebook_handle=True)

In [11]:
interact(update, i=(0, 10))

interactive(children=(IntSlider(value=0, description='i', max=10), Output()), _dom_classes=('widget-interact',…

<function __main__.update(i=0)>

In [12]:
%mpdaf

Numpy 1.15.4
Astropy 3.2.dev23560
MPDAF 3.1.dev2783


In [13]:
#s = Source.from_file('/home/simon/data/UDF/udf_origin_00223.fits')
s = Source.from_file('/home/simon/data/UDF/udf10_c031_e021/udf_udf10_00001.fits')

In [14]:
#s.info()

In [15]:
', '.join(sorted(s.images))

'3DHST_F140W, 3DHST_SEG, 3DHST_SPEC, 3DHST_WHT, HST_F105W, HST_F125W, HST_F140W, HST_F160W, HST_F225W, HST_F275W, HST_F336W, HST_F435W, HST_F606W, HST_F775W, HST_F775W_E, HST_F814W, HST_F850LP, HST_SEGMAP, MASK_OBJ, MASK_SKY, MUSE_EXPMAP, MUSE_WHITE, NB_H10, NB_H11, NB_H8, NB_H9, NB_HBETA, NB_HDELTA, NB_HEPSILON, NB_HGAMMA, NB_NEIII3869, NB_OIII4959, NB_OIII5007, NB_SUMOII3727'

In [16]:
', '.join(sorted(s.spectra))

'3DHST_SENS, MUSE_PSF, MUSE_PSF_SKYSUB, MUSE_SKY, MUSE_TOT, MUSE_TOT_SKYSUB, MUSE_WHITE, MUSE_WHITE_SKYSUB, PFIT_CONT, PFIT_SPEC'

In [19]:
from astropy.visualization import ZScaleInterval, PercentileInterval, MinMaxInterval
from bokeh.models import LogColorMapper, LogTicker, ColorBar, LinearColorMapper

def plot_image(im, size=(350, 350), title=None, colorbar=True, palette="Viridis256", scale='minmax', axis=True, x_range=None, y_range=None):
    if scale == 'zscale':
        interval = ZScaleInterval()
    elif scale == 'percentile':
        interval = PercentileInterval(99)
    elif scale == 'minmax':
        interval = MinMaxInterval()

    vmin, vmax = interval.get_limits(im.data)
    color_mapper = LinearColorMapper(palette=palette, low=vmin, high=vmax)
    if x_range is None:
        x_range = (0, im.shape[0])
    if y_range is None:
        y_range = (0, im.shape[1])

    p = figure(plot_width=size[0], plot_height=size[1], 
               x_range=x_range, y_range=y_range, title=title)
    p.image(image=[im.data], x=[0], y=[0], dw=[im.shape[0]], dh=[im.shape[1]], 
            color_mapper=color_mapper)
    p.grid.visible = False
    p.axis.visible = axis

    if colorbar:
        color_bar = ColorBar(color_mapper=color_mapper, #ticker=LogTicker(),
                             label_standoff=12, border_line_color=None, location=(0,0))
        p.add_layout(color_bar, 'right')

    return p

In [20]:
def images(s, size=(350, 350)):
    images = {
        'MUSE_WHITE': {'scale': 'zscale'},
        'MUSE_EXPMAP': {'scale': 'minmax', 'palette': 'Greys256', 'link': 'MUSE_WHITE'},
        'MASK_OBJ': {'scale': 'minmax', 'palette': 'Greys256', 'link': 'MUSE_WHITE'}, 
        'HST_F606W': {'scale': 'percentile'}, 
        'HST_F775W': {'scale': 'percentile', 'link': 'HST_F606W'}
    }
    for name, params in images.items():
        im = s.images[name]
        kw = dict(size=size, title=name, colorbar=False,
                  scale=params['scale'], palette=params.get('palette', 'Viridis256'))
        if params.get('link') is not None:
            kw['x_range'] = images[params['link']]['plot'].x_range
            kw['y_range'] = images[params['link']]['plot'].y_range
            
        s2 = plot_image(im, **kw)

        if name == 'MUSE_WHITE':
            y, x = im.wcs.sky2pix((s.DEC, s.RA))[0]
            s2.circle(y, x, size=4, color='red', alpha=1)
        
        params['plot'] = s2

    return [im['plot'] for im in images.values()]

In [21]:
def spectrum(s, size=(800, 350)):
    s1 = figure(plot_width=size[0], 
                plot_height=size[1],
                title=f"Origin source #{s.ID}")

    snames = ['MUSE_TOT_SKYSUB', 'MUSE_PSF_SKYSUB', 'MUSE_WHITE_SKYSUB']
    palette = Category10[8]
    
    for i, sname in enumerate(snames):
        sp = s.spectra[sname]
        line = s1.line(sp.wave.coord(), sp.data, legend=sname, color=palette[i])
        if i > 0:
            line.visible = False

    #s1.y_range = Range1d(0, spmax + 10)
    #s1.extra_y_ranges = {"var": Range1d(start=0, end=sp.var.max())}
    #s1.line(sp.wave.coord(), sp.var, legend='var', line_color='gray', line_alpha=0.6)#, y_range_name="var")
    #s1.add_layout(LinearAxis(y_range_name="var"), 'left')

    s1.legend.location = "top_left"
    s1.xaxis.axis_label = f'Wavelength ({sp.wave.unit})'
    s1.yaxis.axis_label = f'Flux ({sp.unit})'
    
    s1.legend.click_policy = "hide"
    
    return [s1]

In [22]:
#p.grid.grid_line_alpha = 0
#p.ygrid.band_fill_color = "olive"
#p.ygrid.band_fill_alpha = 0.1

#output_file('output.html')
l = gridplot([
    images(s, size=(250,250)), 
    spectrum(s, size=(750, 300))
])

#l = layout([
#    images(size=(200,200)),
#    [s1]
#], sizing_mode='fixed')

show(l)
#plot = show(p, notebook_handle=True)